In [108]:
!pip install firebase

In [109]:
pip install requests beautifulsoup4

In [110]:
!pip install nltk



In [111]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from firebase import firebase
from IPython.display import HTML, display
from google.colab import output
from nltk.stem import PorterStemmer
import re
import requests
from bs4 import BeautifulSoup
import json
import random
import string
from urllib.parse import urlparse, urljoin
from collections import deque
import base64
from io import BytesIO
import matplotlib.pyplot as plt
import time
import datetime
from collections import defaultdict
import math
import nltk
from nltk.chat.util import Chat, reflections


#Connect to Firebase
FBconn = firebase.FirebaseApplication('https://cloudproject-1bbde-default-rtdb.firebaseio.com/', None)

In [112]:
# Path to your Firebase Admin SDK service account key
service_account_key = 'cloudproject-1bbde-firebase-adminsdk-849hb-47666b58c8.json' # This file must be added loaclly to Google Colab

# Initialize Firebase Admin SDK
if not firebase_admin._apps:
    cred = credentials.Certificate(service_account_key)
    firebase_admin.initialize_app(cred)

# Get a Firestore client
db = firestore.client()

In [113]:
def query_word(word):
    # Reference the collection where words are stored
    words_collection = db.collection('inverted_index')

    # Query for the document matching the given word
    doc_ref = words_collection.document(word)
    doc = doc_ref.get()

    if doc.exists:
        # print(f"Data for word '{word}': {doc.to_dict()}")
        return doc.to_dict()
    else:
        # print(f"No data found for word '{word}'.")
        return None

In [116]:
def remove_stop_words(index):
  stop_words = {'a', 'an', 'and', 'are', 'as', 'at', 'be', 'but', 'by', 'for', 'if', 'in', 'into', 'is', 'it', 'no', 'not', 'of', 'on', 'or', 'such', 'that', 'the', 'their', 'then',
                'there', 'these', 'they', 'this', 'to', 'was', 'will', 'with'}

  words = index.split()

  # Remove stop words from the list of words
  filtered_words = [word for word in words if word.lower() not in stop_words]

  # Join the filtered words back into a string
  result_str = ' '.join(filtered_words)
  return result_str

In [117]:
def apply_stemming(word):
  stemmer = PorterStemmer()
  stemmed_word = stemmer.stem(word)
  return stemmed_word

In [121]:
def extract_link_title(link):
  # Fetch the webpage content
  response = requests.get(link)

  # Use BeautifulSoup to parse the HTML content
  soup = BeautifulSoup(response.text, 'html.parser')

  if soup.title:
    title = soup.title.text
    result = [{"title": title, "link": link}]
  else:
    title = link
    result = [{"title": link, "link": link}]


  # Find the title tag and get its text
  return title

In [123]:
def update_index(words_list, index):
  # This function updates the index, if the user type a word that not in index, this function will update the index and add the word with the relavent links
  temp_index = []
  temp_index2 = []
  for word in words_list:
    words_list = query_word(word)
    if words_list:
      if words_list['urls'] not in temp_index:
        temp_index += words_list['urls']
        temp_index2 += words_list['urls']
        index[word] = temp_index
    temp_index = []
  return index, temp_index, temp_index2


def create_index(index):
  # This function initialize the index with some words (helps in Time complexity of searching)
  words = ["ai", "cloud", "ibm", "devops", "industry", "servers", "security", "application", "business", "databases", "financial", "public", "private"]
  buffer = []
  index, buffer, buffer = update_index(words, index)
  return index

In [124]:
# Initialize Firebase Admin SDK
cred = credentials.Certificate('cloudproject-1bbde-firebase-adminsdk-849hb-47666b58c8.json') # Update the path to your downloaded JSON file
# firebase_admin.initialize_app(cred)

# Get a Firestore client
# db = firestore.client()

def upload_index_to_firestore(index):
    """
    Uploads or updates the index in Firestore.

    :param index: The index to upload or update, a dictionary where keys are words and values are arrays of links.
    """
    # Reference the collection in Firestore
    index_collection = db.collection('index')

    for term, urls in index.items():
        # Check if the document for the term exists in Firestore
        doc_ref = index_collection.document(term)
        doc = doc_ref.get()

        if doc.exists:
            # If the document exists, update the URLs with the new values
            existing_urls = doc.to_dict().get('urls', [])
            new_urls = [url for url in urls if url not in existing_urls]
            updated_urls = existing_urls + new_urls
            doc_ref.update({'urls': updated_urls})
        else:
            # If the document doesn't exist, create a new document with the URLs
            doc_ref.set({'urls': urls})

In [126]:
def extract_index_data_from_firestore():
    # Extract index data from Firestore
    index_data = {}  # Initialize an empty dictionary to store index data
    index_ref = firestore.client().collection('index')
    docs = index_ref.stream()  # Retrieve all documents from the 'index' collection

    for doc in docs:
        word = doc.id  # Get the word (document ID)
        links = doc.to_dict()  # Get the links associated with the word
        index_data[word] = links  # Add the word and its links to the dictionary
    return index_data





def generate_index_table_rows(index_data):
    rows = ""
    for word, data in index_data.items():
        # Get the links associated with the word
        links = data.get('urls', [])  # If 'urls' key doesn't exist, default to an empty list

        # Remove duplicates from links
        unique_links = list(set(links))

        # Create HTML for unique links
        links_html = "<br>".join([f"<a href='{link}'>{link}</a>" for link in unique_links])

        # Create a row for each word and its associated unique links
        rows += f"<tr><td>{word}</td><td>{links_html}</td></tr>"
    return rows

In [127]:
# Initialize Firebase Admin SDK with credentials
# cred = credentials.Certificate("cloudproject-1bbde-firebase-adminsdk-849hb-47666b58c8.json")
# firebase_admin.initialize_app(cred)

# Initialize Firestore database
# db = firestore.client()

def remove_word_from_firestore(word):
    try:
        # Delete the document corresponding to the word
        db.collection('index').document(word).delete()

    except Exception as e:
        print("Error removing word from Firestore:", e)

# Register the Python function to be invoked from JavaScript
output.register_callback('notebook.removeWord', remove_word_from_firestore)

In [128]:
def handle_new_item(new_word, new_link):
    # Update the index with the new word and link
    index = extract_index_data_from_firestore()
    if new_word in index:
        index[new_word].append(new_link)
    else:
        index[new_word] = [new_link]

    # Upload the updated index to Firestore
    upload_index_to_firestore(index)

    # Refill the table with the new index
    index_data = extract_index_data_from_firestore()
    index_table_rows = generate_index_table_rows(index_data)
    javascript_refill_table = f"""
        <script>
          updateIndexTable(`{index_table_rows}`);
        </script>
    """
    display(HTML(javascript_refill_table))

# Register the Python function to be invoked from JavaScript
output.register_callback('notebook.handleNewItem', handle_new_item)

In [129]:
def handle_new_link(selected_word, new_link):
    """
    Handles the selected word and new link.

    :param selected_word: The selected word.
    :param new_link: The new link.
    """
    # Update Firestore database
    doc_ref = db.collection('index').document(selected_word)
    doc = doc_ref.get()

    if doc.exists:
        # If the document exists, update the URLs with the new link
        existing_urls = doc.to_dict().get('urls', [])
        updated_urls = existing_urls + [new_link]
        doc_ref.update({'urls': updated_urls})
    else:
        # If the document doesn't exist, create a new document with the new link
        doc_ref.set({'urls': [new_link]})

    # Refill the table with the updated index data
    index_data = extract_index_data_from_firestore()
    index_table_rows = generate_index_table_rows(index_data)
    javascript_refill_table = f"""
      <script>
        updateIndexTable(`{index_table_rows}`);
      </script>
    """
    display(HTML(javascript_refill_table))
# Register the Python function to be invoked from JavaScript
output.register_callback('notebook.handleNewLink', handle_new_link)


In [130]:
def display_statistics(index_data=None):
    if index_data is None:
        # Fetch index data from Firestore if not provided
        index_data = extract_index_data_from_firestore()

    links = []
    for value in index_data.values():
        urls = value.get('urls', [])
        links.extend(urls)

    unique_links = list(set(links))
    link_count = {link: links.count(link) for link in unique_links}
    sorted_link_count = dict(sorted(link_count.items(), key=lambda x: x[1], reverse=True))

    labels = list(sorted_link_count.keys())
    sections = list(sorted_link_count.values())
    total_sections = sum(sections)

    # Combine single occurrences into 'Others' category if there are more than 10 labels
    if len(labels) > 10:
        single_occurrences = sum(1 for count in sections if count == 1)
        labels = labels[:10] + ['Others (each with 1 occurrence)']
        sections = sections[:10] + [single_occurrences]

    fig, ax = plt.subplots(figsize=(10, 8))
    patches, texts, _ = ax.pie(sections, labels=labels, startangle=90, autopct='%1.1f%%')
    ax.set_title('Links Occurrences')
    ax.legend(loc='best', bbox_to_anchor=(1, 1))
    ax.axis('equal')

    # Save the plot to a bytes buffer in PNG format
    buffer = BytesIO()
    plt.savefig(buffer, format='png', bbox_inches="tight")
    plt.close(fig)
    buffer.seek(0)
    # Encode the PNG image to base64 string
    image_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')
    image_html = f'<img src="data:image/png;base64,{image_base64}" />'

    # Generate HTML content for the paragraph and the embedded image
    paragraph_html = '<p>This pie chart displays the occurrences of links indexed.</p>'
    combined_html = paragraph_html + image_html

    return combined_html

output.register_callback('notebook.displayStatistics', display_statistics)

In [131]:
# Initialize Firebase Admin SDK
# cred = credentials.Certificate('cloudproject-1bbde-firebase-adminsdk-849hb-47666b58c8.json') # Update the path to your downloaded JSON file
# firebase_admin.initialize_app(cred)

# Get a Firestore client
# db = firestore.client()

def login(username, password):
    """
    Checks if the provided username and password are valid.

    :param username: The username entered by the user.
    :param password: The password entered by the user.
    """
    if not username or not password:
        # Either username or password is empty
        message = "<p style='color: red;'>You have to enter Username and Password</p>"
        output.eval_js(f"document.getElementById('login-message').innerHTML = `{message}`;")
        return

    # Query Firestore to check if the user exists and if the password is correct
    user_ref = db.collection('users').document(username)
    user_doc = user_ref.get()

    if user_doc.exists:
        # User exists, check if the password matches
        user_data = user_doc.to_dict()
        if user_data['password'] == password:
          # Check if the user is already logged in
          if user_data.get('isLoggedIn', 0) == 0:
            # Update the isLoggedIn flag to 1
            user_ref.update({'isLoggedIn': 1})
            message = "<p style='color: green;'>Welcome!</p>"
            output.eval_js(f"document.getElementById('login-message').innerHTML = `{message}`;")

            time.sleep(1)



            loginTime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            user_id = user_data.get('user_id')

            output.eval_js("showContent('home-content');")
            if user_id:

              user_sessions_ref = db.collection('user_sessions').document(user_id)
              user_sessions_ref.update({'loginTime': firestore.ArrayUnion([loginTime])})

          else:
            # User already logged in
            message = "<p style='color: red;'>User already logged in!</p>"
            output.eval_js(f"document.getElementById('login-message').innerHTML = `{message}`;")
            # Update the isLoggedIn flag to 0 to allow logging in again
            user_ref.update({'isLoggedIn': 0})

        else:
          message = "<p style='color: red;'>Incorrect Password!</p>"
          output.eval_js(f"document.getElementById('login-message').innerHTML = `{message}`;")


    else:
        message = "<p style='color: red;'>User doesn't exists</p>"
        output.eval_js(f"document.getElementById('login-message').innerHTML = `{message}`;")

# Register the login function to be callable from JavaScript
output.register_callback('notebook.login', login)


In [132]:
def create_line_plot_with_markers(user_id):
  # Initialize Firestore client
  # db = firestore.Client()

  # Reference the user_sessions collection
  user_sessions_ref = db.collection('user_sessions').document(user_id)

  # Get the document data
  user_sessions_data = user_sessions_ref.get().to_dict()
  if user_sessions_data:
    user_times = user_sessions_data.get('userTime')


In [133]:
def logout():
    # Reference the collection in Firestore
    users_collection = db.collection('users')
    user_sessions_collection = db.collection('user_sessions')

    # Query users who are logged in
    query = users_collection.where('isLoggedIn', '==', 1).get()
    logged_in_users = [doc.id for doc in query]

    logoutTime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Loop through the logged in users and update their isLoggedIn field to 0
    for user_name in logged_in_users:

      user_doc = users_collection.document(user_name).get()
      if user_doc.exists:
        user_id = user_doc.get('user_id')
        user_sessions_ref = db.collection('user_sessions').document(user_id)
        user_sessions_ref.update({'logoutTime': firestore.ArrayUnion([logoutTime])})

      user_ref = users_collection.document(user_name)
      user_ref.update({'isLoggedIn': 0})


      login_times = get_login_times(user_id)
      logout_times = get_logout_times(user_id)
      calculate_total_seconds(login_times, logout_times, user_id)

output.register_callback('notebook.logout', logout)


In [134]:
def signup(username, password, security_question):
    """
    Handles the signup process and saves user data to Firestore.

    :param username: The username entered by the user.
    :param password: The password entered by the user.
    :param security_question: The security question entered by the user.
    """

    # Check if username or password is missing
    if not username or not password:
      message = "<p style='color: red;'>You have to enter Username and Password</p>"
      output.eval_js(f"document.getElementById('signup-message').innerHTML = `{message}`;")
      return

    user_id = generate_user_id() # Getting id for the user

    # Reference the collection in Firestore
    users_collection = db.collection('users')

    # Set the document with the username as the document ID
    user_doc = users_collection.document(username)


    # Check if the username already exists
    existing_user = users_collection.document(username).get()
    if existing_user.exists:
      message = "<p style='color: red;'>Username already exists. Please try another one.</p>"
      output.eval_js(f"document.getElementById('signup-message').innerHTML = `{message}`;")
      return
    # Reference the user_sessions collection
    user_sessions_collection = db.collection('user_sessions')

    # Create a new document for the user session
    user_session_doc = user_sessions_collection.document(user_id)

    try:


      # Initialize the session fields
      user_session_doc.set({
          'queries': [],
          'answers': [],
          'created_at': datetime.datetime.fromtimestamp(time.time())
    })
    except Exception as e:
      message = "<p style='color: red;'>Something went wrong, please try again later!</p>"
      output.eval_js(f"document.getElementById('signup-message').innerHTML = `{message}`;")
      return


    # Set user data in the document
    try:
      user_doc.set({
          'user_id': user_id,
          'username': username,
          'password': password,
          'security_question': security_question,
          'isLoggedIn': 0  # Initialize isLoggedIn to 0
      })
      message = "<p style='color: green;'>Signup successful!</p>"
      output.eval_js(f"document.getElementById('signup-message').innerHTML = `{message}`;")
      time.sleep(1)
      output.eval_js("showContent('login-content');")

    except Exception as e:
      message = "<p style='color: red;'>Something went wrong, please try again later!</p>"
      output.eval_js(f"document.getElementById('signup-message').innerHTML = `{message}`;")
      return


output.register_callback('notebook.signup', signup)

In [135]:
# Initialize Firebase Admin SDK
# cred = credentials.Certificate('cloudproject-1bbde-firebase-adminsdk-849hb-47666b58c8.json')
# firebase_admin.initialize_app(cred)

# Get a Firestore client
# db = firestore.client()

def generate_user_id():
  # This function generates user id


  # Reference the document containing the latest_id
  id_doc_ref = db.collection('user_id').document('ID')
  # Retrieve the current value of latest_id
  id_doc = id_doc_ref.get()
  latest_id_str = id_doc.get('latest_id')
  if latest_id_str is not None:
    # Convert the latest_id to an integer, increment it by 1, and format it as a 3-digit string
    latest_id_int = int(latest_id_str)
    next_id_int = latest_id_int + 1
    next_id_str = str(next_id_int).zfill(3) # Pad with leading zeros to ensure 3 digits
    id_doc_ref.update({'latest_id': next_id_str})
    return next_id_str
  else:
    return None




In [136]:
def get_login_times(user_id):
    # Access user_sessions collection and retrieve last login time
    user_sessions_ref = db.collection('user_sessions').document(user_id)
    user_sessions_doc = user_sessions_ref.get()
    login_times = user_sessions_doc.get('loginTime')
    if login_times:
        return login_times
    else:
        return None



In [137]:
def get_logout_times(user_id):
  # Access user_sessions collection and retrieve logout times
  user_sessions_ref = db.collection('user_sessions').document(user_id)
  user_sessions_doc = user_sessions_ref.get()
  logout_times = user_sessions_doc.get('logoutTime')
  if logout_times:
    return logout_times
  else:
    return None


In [138]:
def calculate_total_seconds(login_times, logout_times, user_id):
    # Initialize a dictionary to store total seconds for each date
    total_seconds_dict = defaultdict(int)

    # Iterate through each login and logout time
    for login_time, logout_time in zip(login_times, logout_times):
        # Extract date part from login and logout times
        login_date = login_time.split()[0]
        logout_date = logout_time.split()[0]

        # If login and logout dates are the same
        if login_date == logout_date:
            # Calculate time difference in seconds
            time_difference = (
                datetime.datetime.strptime(logout_time, '%Y-%m-%d %H:%M:%S') -
                datetime.datetime.strptime(login_time, '%Y-%m-%d %H:%M:%S')
            ).total_seconds()

            # Update total seconds for the date
            total_seconds_dict[login_date] += time_difference
        else:
            # Calculate time difference until end of day for login time
            time_difference = (
                datetime.strptime(login_date + ' 23:59:59', '%Y-%m-%d %H:%M:%S') -
                datetime.strptime(login_time, '%Y-%m-%d %H:%M:%S')
            ).total_seconds()

            # Update total seconds for the login date
            total_seconds_dict[login_date] += time_difference

            # Calculate time difference from start of day for logout time
            time_difference = (
                datetime.strptime(logout_time, '%Y-%m-%d %H:%M:%S') -
                datetime.strptime(logout_date + ' 00:00:00', '%Y-%m-%d %H:%M:%S')
            ).total_seconds()

            # Update total seconds for the logout date
            total_seconds_dict[logout_date] += time_difference


    # Access the user_sessions document for the user_id
    user_sessions_ref = db.collection('user_sessions').document(user_id)

    # Update the userTime field with the total_seconds_dict
    user_sessions_ref.update({'userTime': total_seconds_dict})




In [139]:
def get_user_id():
    # Reference the users collection in Firestore
    users_collection = db.collection('users')

    # Query users where isLoggedIn is equal to 1
    query = users_collection.where('isLoggedIn', '==', 1).get()

    # Check if any users are logged in
    if not query:
        return None  # No users are logged in

    # Extract the user_id of the first user found (assuming only one user can be logged in at a time)
    for doc in query:
        # Return the value of the 'user_id' field
        return doc.get('user_id')

# Register the function as a callback
output.register_callback('notebook.get_user_id', get_user_id)


In [140]:
def count_links():
  # Define a defaultdict to count the occurrences of each link
  link_count = defaultdict(int)

  # Reference to the "index" collection
  index_ref = db.collection("index")


  # Get all documents from the "index" collection
  index_docs = index_ref.get()

  # Iterate over each document in the "index" collection
  for doc in index_docs:
    # Get the array of links from the document data
    links = doc.to_dict().get("urls")

    # Iterate over each link in the array and count its occurrence
    for link in links:
        link_count[link] += 1

  return dict(link_count)

In [141]:
def display_user_times(user_id):
    # Retrieve data from Firestore
    index = 0
    all_documents = get_document_ids("user_sessions")
    for i in range(len(all_documents)):
        if user_id == all_documents[i]:
            index = i

    user_sessions_ref = db.collection('user_sessions').document(all_documents[index])
    user_sessions_doc = user_sessions_ref.get()
    user_time_data = user_sessions_doc.get('userTime')

    # Check if the document exists
    if not user_sessions_doc.exists:
        return f"<p>User document with ID '{all_documents[index]}' not found.</p>"

    # Extract data from the document
    user_data = user_sessions_doc.to_dict()
    user_time_data = user_data.get('userTime')

    if not user_time_data:
        return "<p>No user time data found.</p>"

    # Get the last seven dates available in the dictionary
    last_seven_dates = sorted(list(user_time_data.keys()), reverse=True)[:7]

    x_values = []
    y_values = []

    for day in last_seven_dates:
        x_values.append(day)
        # Convert seconds to hours
        y_values.append(user_time_data[day] / 3600)  # 3600 seconds in an hour

    # Plot the data
    plt.figure(figsize=(10, 6))
    plt.plot(x_values, y_values, marker='o', linestyle='-')
    plt.title('User Time in the Last 7 Days')
    plt.xlabel('Date')
    plt.ylabel('Hours')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()

    # Save the plot to a bytes buffer in PNG format
    buffer = BytesIO()
    plt.savefig(buffer, format='png', bbox_inches="tight")
    plt.close()
    buffer.seek(0)

    # Encode the PNG image to base64 string
    image_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')
    image_html = f'<img src="data:image/png;base64,{image_base64}" />'

    # Count links
    link_counts = count_links()

    # Extract the links and counts from the dictionary
    links = list(link_counts.keys())
    counts = list(link_counts.values())

    # Plotting the bar chart
    plt.figure(figsize=(10, 6))
    plt.bar(links, counts)
    plt.xlabel('Links')
    plt.ylabel('Counts')
    plt.title('Link Counts')
    plt.xticks(rotation=90)  # Rotate x-axis labels for better readability

    # Save the link counts plot to a bytes buffer in PNG format
    buffer_link_counts = BytesIO()
    plt.savefig(buffer_link_counts, format='png', bbox_inches="tight")
    plt.close()
    buffer_link_counts.seek(0)

    # Encode the PNG image to base64 string
    image_base64_link_counts = base64.b64encode(buffer_link_counts.getvalue()).decode('utf-8')
    image_html_link_counts = f'<img src="data:image/png;base64,{image_base64_link_counts}" />'

    # Return both plots
    return image_html, image_html_link_counts

output.register_callback('notebook.displayUserStatistics', display_user_times)


In [142]:
# Get a list of document IDs from a collection
def get_document_ids(collection_name):
    # Reference the collection
    collection_ref = db.collection(collection_name)

    # Query all documents
    docs = collection_ref.stream()

    # Extract document IDs
    document_ids = [doc.id for doc in docs]

    return document_ids



In [143]:
def count_query_occurrences(link, user_query):
    # Retrieve the content of the link
    response = requests.get(link)
    if response.status_code == 200:
        content = response.text.lower()  # Convert content to lowercase for case-insensitive matching
        # Split the user query into individual words
        query_words = user_query.lower().split()
        # Initialize the dictionary to store word counts
        word_counts = {}
        # Count occurrences of each word from the user query in the content
        for word in query_words:
            count = content.count(word)
            word_counts[word] = count
        # Find the largest counter among the word counts
        max_count = max(word_counts.values())
        return max_count
    else:
        return 0

In [144]:
def calculate_rank(occurrences):
    # Apply a modified logarithmic transformation to the occurrence counts
    transformed_occurrences = [math.log(occurrence + 1) for occurrence in occurrences]

    # Find the maximum transformed occurrence count
    max_transformed_occurrence = max(transformed_occurrences)

    # Calculate rank for each page using the transformed occurrence counts
    rank = [(transformed_occurrence / (max_transformed_occurrence + 1)) for transformed_occurrence in transformed_occurrences]

    return rank

In [145]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Define some patterns and responses related to Alibaba Cloud terms
patterns = [
    (r'hi|hello|hey', ['Hello!', 'Hi there!', 'Hey!']),
    (r'how are you?', ['I\'m good, thank you!', 'I\'m doing well, thanks for asking.']),
    (r'what is your name?', ['You can call me ClChat.', 'I go by the name ClChat.']),
    (r'your name (.*)', ['You can call me ClChat.', 'I go by the name ClChat.']),
    (r'give me good cloud websites?', ['Sure, here are some websites: www.alibabacloud.com. You might also find these useful: www.aws.amazon.com, www.azure.microsoft.com, www.googlecloud.com']),
    (r'what is (S|s)aaS\?', ['SaaS stands for Software as a Service. It is a software delivery model where software is hosted on cloud and accessed via the internet.']),
    (r'(what are|what is) (P|p)aaS\?', ['PaaS stands for Platform as a Service. It provides a platform allowing customers to develop, run, and manage applications without dealing with infrastructure.']),
    (r'(what are|what is) (I|i)aaS\?', ['IaaS stands for Infrastructure as a Service. It provides virtualized computing resources over the internet, such as virtual machines and storage.']),
    (r'what is (I|i)BM (C|c)loud\?', ['IBM Cloud is the enterprise cloud for even the most mission critical workloads. With over 170+ public cloud solutions, our platform is helping clients mitigate 3rd and 4th party risk, increase time to value, while reducing total cost of ownership (TCO).']),    (r'(what are|what is) (P|p)rivate (C|c)loud\?', ['A private cloud is a cloud computing environment dedicated solely to one organization, providing greater control and security.']),
    (r'(what are|what is) (P|p)ublic (C|c)loud\?', ['A public cloud is a cloud computing environment shared by multiple organizations, offering scalability and cost-effectiveness.']),
    (r'(what are|what is) (H|h)ybrid (C|c)loud\?', ['A hybrid cloud is a combination of public and private clouds, allowing data and applications to be shared between them.']),
    (r'(describe|explain) (P|p)rivate (C|c)loud\?', ['A private cloud is a cloud computing model that offers a secure and dedicated environment for a single organization. It provides greater control over resources and data, suitable for businesses with strict security and compliance requirements.']),
    (r'(describe|explain) (P|p)ublic (C|c)loud\?', ['A public cloud is a cloud computing model where resources are shared among multiple users over the internet. It offers scalability, flexibility, and cost-effectiveness, making it ideal for startups, small businesses, and organizations with fluctuating workloads.']),
    (r'(describe|explain) (H|h)ybrid (C|c)loud\?', ['A hybrid cloud is a cloud computing environment that combines elements of both public and private clouds. It allows data and applications to be shared between them, offering flexibility, scalability, and the ability to meet specific business requirements.']),
    (r'(.*)(kinds of|types of) (C|c)louds(.*)', ['There are various types of clouds, such as public, private, and hybrid clouds. Would you like to know more about a specific type?']),
    (r'(explain|describe) (C|c)louds(.*)', ['Sure, I can explain. Could you please specify which type of cloud you want to know about?']),
    (r'(how does|did|had|have) (you|your) (page rank|rank) (calculated|calculate)\?', ['My page rank is calculated using a modified logarithmic transformation.']),  # Added pattern and response for page rank calculation
    (r'(show me|what is) my index\?', ["According to my last update, i can inform you with the following information: Your index includes: 'add', 'ai', 'application', 'business', 'cloud', 'complex', 'databases', 'devops', 'financial', 'free', 'ibm', 'industry', 'public', 'security'.\n This information provided according to my last knowledge and update!"]),  # Added pattern and response for showing index data
    (r'(what is page indexing|what is indexing|explain to me about indexing)\?', ['Web indexing is the process of collecting and storing information from web pages to create a searchable index that can be used by search engines to provide relevant results to users\' queries.']),  # Added pattern and response for indexing
    (r'(what is|how is) the (page|page\'s) (ranking|rank) for (.*)', ['The ranking of the page for "{}" is calculated based on the number of occurrences of the query in the page content.'.format('\1')]),
    (r'(explain|describe) about your search engine', ['Our search engine is a user-friendly interface that provides valuable and valid information. It has a large database and the ability to expand further, providing user and general statistics. Our database is secure and continually updated.']),
]

# Create a chatbot
chatbot = Chat(patterns, reflections)

def get_chatbot_response(user_input):
  return chatbot.respond(user_input)

output.register_callback('notebook.get_chatbot_response', get_chatbot_response)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [146]:
html = """
  <!DOCTYPE html>
  <html lang="en">
    <head>
      <meta charset="UTF-8" />
      <meta name="viewport" content="width=device-width, initial-scale=1.0" />
      <link
        rel="stylesheet"
        href="https://cdn.jsdelivr.net/npm/bootstrap@4.3.1/dist/css/bootstrap.min.css"
        integrity="sha384-ggOyR0iXCbMQv3Xipma34MD+dH/1fQ784/j6cY/iJTQUOhcWr7x9JvoRxT2MZw1T"
        crossorigin="anonymous"
      />


      <link rel="stylesheet" href="app.css" />
      <title>ClSearch</title>
      <style>
        body {
          background-color: #e0f2f1; /* Light blue background color */
        }

        .navbar {
          background-color: #ffffff; /* White background color for the navbar */
          box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.1),
            0 2px 4px -1px rgba(0, 0, 0, 0.06); /* Add box shadow for a floating effect */
        }

        .navbar img {
          max-height: 40px; /* Set the maximum height for the logo */
          margin-right: 10px;
        }

        .active-cyan-2 input.form-control[type="text"]:focus:not([readonly]) {
          border-bottom: 1px solid #4dd0e1;
          box-shadow: 0 1px 0 0 #4dd0e1;
        }
        .active-cyan input.form-control[type="text"] {
          border-bottom: 1px solid #4dd0e1;
          box-shadow: 0 1px 0 0 #4dd0e1;
        }
        .active-cyan .fa,
        .active-cyan-2 .fa {
          color: #4dd0e1;
        }

        .content-section {
          display: none; /* Hide all content sections by default */
        }

        .active-content {
          display: block; /* Display the active content section */
        }

        .flex-container{
          display: flex;
          justify-content: space-between;
          align-items: flex-start;
        }

        .table-container{
          flex:1;
        }

        .button-container{
          margin-left: 20px;
        }

        .table-container{
          border: 1px solid #ccc;
          padding: 10px;
          margin: auto;
          width: 80%;
        }


        .button-container{
          border: 1px solid #ccc;
          padding: 10px;
        }

        #index-table-body tr:hover{
          cursor: pointer;
        }

        #index-table-body tr.clicked{
          background-color: rgba(0, 0, 0, 0.1);
          transition: background-color 0.3s ease;
        }

        .btn-unstyled{
          background: none;
          border: none;
          padding: 0;
          font: inherit;
          cursor: pointer;
          color: #007bff;
          text-decoration: underline;
        }

        .btn-unstyled:hover{
          color: #0056b3;
          text-decoration: underline;
        }

        .dropdown-menu {
          position: absolute;
          right: 0;
          left: auto;
          top: 100%;
          min-width: 100%;
          white-space: nowrap;
        }

        .charts-container{
          display: flex;
          flex-direction: column;
          align-items: center;
          justify-content: space-between;
        }

        .chat-container{
          max-width: 800px;
          margin: 20px auto;
          background-color: #fff;
          border-radius: 5px;
          box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
          overflow: hidden;

        }

        .chat-box {
          padding: 20px;
          height: 400px;
          overflow-y: scroll;
        }

        .user-msg, .bot-msg {
          margin: 10px 0;
        }

        .user-msg {
          text-align: right;
          color: #333;
        }

        .bot-msg{
          text-align: left;
          color: #666;
        }




      </style>
      <script>
        function search(event) {
          event.preventDefault();
          var inputValue = document.getElementById("search-input").value;
          console.log("Search Input:", inputValue);
          google.colab.kernel.invokeFunction("notebook.query", [inputValue], {});
          //document.getElementById("search-input").value = "";
          document.getElementById("search-input").blur();
          //return false;
        }

        function handleKeyUp(event) {
          if (event.key === "Enter") {
            search(event);
          }
        }

        function showContent(sectionId) {
          // Hide all content sections
          var sections = document.getElementsByClassName("content-section");
          for (var i = 0; i < sections.length; i++) {
            sections[i].classList.remove("active-content");
          }

          // Display the selected content section
          document.getElementById(sectionId).classList.add("active-content");
        }

        function resetResultContent() {
          var resultsContent = document.getElementById("results-content");
          resultsContent.innerHTML = `<div class="text-start display-5 m-5"><span style="color: #00695c; font-style: italic">Search Results</span></div>`;
        }

        function updateResultsContent(results, ranks) {
          //Display the results in the "Search Results" section
          //var resultContent = document.getElementById("results-content");
          //resultsContent.innerHTML = results;

          // Create a new div element for the result
          var resultDiv = document.createElement("div");

          // Set styling for the result
          resultDiv.style.textAlign = "left";
          resultDiv.style.color = "red";
          resultDiv.style.marginLeft = "20px";
          resultDiv.style.fontSize = "20px"; // Adjust the font size as needed

          var note = document.createElement("p");
          note.textContent = "Note: The number in parentheses represents the occurrences of the query within the page, and the rank is calculated using a modified logarithmic transformation.";
          note.style.color = "#808080";


          // Start building the HTML content
          html_content = "<ul>";

          // # Loop through the search results for "cloud" and add each as a list item
          //for (const object of results) {
            //html_content += `<li><a href="${object["link"]}" target="_blank">${object["title"]}</a> <span style="color: gray; font-size: 14px;">(Occurrences: ${object["occurrences"]})</span></li>`;
            //console.log(object);
          //}

          for (let i = 0; i < results.length; i++) {
            const object = results[i];
            const rank = ranks[i];
            html_content += `<li><a href="${object["link"]}" target="_blank">${object["title"]}</a> <span style="color: gray; font-size: 14px;">(Occurrences: ${object["occurrences"]} - Rank: ${rank.toFixed(4)})</span></li>`;
          }

          // Close the list and div tag
          html_content += "</ul>";

          // Set the content of the div to the results
          resultDiv.innerHTML = html_content;

          // Get the "results-content" section
          var resultsContent = document.getElementById("results-content");

          // Append note under the exisitng content
          resultsContent.appendChild(note);

          // Append the resultDiv under the existing content
          resultsContent.appendChild(resultDiv);
        }

        function none_result_content(query) {
          var resultDiv = document.createElement("div");
          resultDiv.style.textAlign = "left";
          resultDiv.style.color = "red";
          resultDiv.style.marginLeft = "20px";
          resultDiv.style.fontSize = "20px"; // Adjust the font size as needed
          html_content = "<ul>";

          html_content += `<li>There is no search result for ${query}</li>
          `

          // Close the list and div tag
          html_content += "</ul>";

          // Set the content of the div to the results
          resultDiv.innerHTML = html_content;

          // Get the "results-content" section
          var resultsContent = document.getElementById("results-content");

          // Append the resultDiv under the existing content
          resultsContent.appendChild(resultDiv);
        }
        function displayNoResults(){
          var resultsContent = document.getElementById("results-content");
          resultsContent.innerHTML = '<div class="text-start display-5 m-5"><span style="color: #00695c; font-style: italic">No results found</span></div>';
        }

        let selectedWord = "";
        document.querySelectorAll("#index-table-body tr").forEach(row => {
          row.addEventListener("click", function() {
            document.querySelectorAll("#index-table-body tr").forEach(row => {
              row.classList.remove("clicked");
            });
            row.classList.add("clicked");
            selectedWord = row.cells[0].textContent;
          });
        });

        document.querySelector(".remove-word-btn").addEventListener("click", function(){
          if (selectedWord) {
            document.querySelectorAll("#index-table-body tr").forEach(row => {
              if (row.cells[0].textContent === selectedWord) {
                row.remove();
                google.colab.kernel.invokeFunction("notebook.removeWord", [selectedWord], {});
              }
            });
            selectedWord = "";
          }
        });



        document.getElementById('addWordBtn').addEventListener('click', function() {
          console.log('Add Word button clicked');
          document.getElementById('addWordSection').style.display = 'block';
        });

        document.getElementById('cancelBtn').addEventListener('click', function() {
          document.getElementById('addWordSection').style.display = 'none';
        });

        document.getElementById('saveWordBtn').addEventListener('click', function() {
          var newWord = document.getElementById('newWord').value;
          var newLink = document.getElementById('newLink').value;

          if (newWord && newLink) {

            google.colab.kernel.invokeFunction('notebook.handleNewItem', [newWord, newLink], {});
            document.getElementById('addWordSection').style.display = 'none';
          } else {
            alert('Please fill in both fields: New Word and New Link.');
          }
        });

        function updateIndexTable(indexTableRows) {
          var indexTableBody = document.getElementById('index-table-body');
          indexTableBody.innerHTML = indexTableRows;
        }


        let selected_word = "";
        document.querySelectorAll("#index-table-body tr").forEach(row => {
          row.addEventListener("click", function() {
            document.querySelectorAll("#index-table-body tr").forEach(row => {
              row.classList.remove("clicked");
            });
            row.classList.add("clicked");
            selected_word = row.cells[0].textContent;
          });
        });

        document.getElementById('addLinkBtn').addEventListener('click', function() {
          console.log('Add Link button clicked');
          console.log('Selected Word:', selected_word);
          document.getElementById('addLinkSection').style.display = 'block';
        });

        document.getElementById('cancelLinkBtn').addEventListener('click', function() {
          document.getElementById('addLinkSection').style.display = 'none';
        });

        document.getElementById('saveLinkBtn').addEventListener('click', function() {
          let newLink = document.getElementById('newLink-addlink').value;
          console.log("FROM SAVELINKBTN", selected_word);

          if (selected_word && newLink) {
            google.colab.kernel.invokeFunction('notebook.handleNewLink', [selected_word, newLink], {});
            console.log('Selected Word:', selected_word);
            console.log('New Link:', newLink);
            document.getElementById('addLinkSection').style.display = 'none';
          } else {
            alert('Please select a word from the table and fill in the New Link field.');
          }
        });

        function showStatistics() {
          google.colab.kernel.invokeFunction('notebook.displayStatistics', [], {})
            .then(function(response) {
              // Extract the HTML content from the Python function response
              var htmlContent = response.data['text/plain'];

              htmlContent = htmlContent.slice(1, -1);

              // Create a new div element to hold the HTML content
              var div = document.getElementById('statistics-chart');
              div.innerHTML = htmlContent;

            }).catch(function(error) {
              console.error('Error:', error);
            });
        }

        function showUserStatistics() {
          google.colab.kernel.invokeFunction('notebook.get_user_id', [], {})
            .then(function(response) {
              var user_id = response.data['text/plain'];
              if (user_id) {
                google.colab.kernel.invokeFunction('notebook.displayUserStatistics', [user_id], {})
                  .then(function(response){
                    var htmlContent = response.data['text/plain'];
                    console.log("here is htmlContent", htmlContent);
                    htmlContent = htmlContent.slice(1, -1);
                    var div = document.getElementById('user-statistics-chart');
                    div.innerHTML = htmlContent;
                  }).catch(function(error) {
                    console.error('Error:', error);
                  });
              } else {
                console.error('No logged-in user found.');
              }
            }).catch(function(error) {
              console.error('Error:', error);
            });

        }




        //document.getElementById("signup-btn").addEventListener("click", signup);
        function signup() {
          // Get user input from the form
          var username = document.getElementById("signup-username").value;
          var password = document.getElementById("signup-password").value;
          var securityQuestion = document.getElementById("signup-security-question").value;

          // Call the Python function using google.colab.kernel.invokeFunction
          google.colab.kernel.invokeFunction('notebook.signup', [username, password, securityQuestion], {});
        }

        //function showLoginForm() {
          //document.getElementById("signup-form-container").style.display = "none";
          //document.getElementById("login-form-container").style.display = "block";
        //}

        function login() {
          //Get user input from form
          var username = document.getElementById("login-username").value;
          var password = document.getElementById("login-password").value;

          //Call the Python function using google.colab.kernel.invokeFunction
          //google.colab.kernel.invokeFunction('notebook.login', [username, password], {});
          google.colab.kernel.invokeFunction('notebook.login', [username, password], {}).then(function(output){
            document.getElementById('login-btn').style.display = 'none';
            document.getElementById('signup-btn').style.display = 'none';
            document.getElementById('logout-btn').style.display = 'block';
            document.getElementById('user-icon-btn').style.display = 'block';
          });

        }

        function toggleDropdown() {
          var dropdown = document.getElementById("user-dropdown");
          if (dropdown.style.display === "none") {
            dropdown.style.display = "block";
          } else {
            dropdown.style.display = "none";
          }
        }

        function logout(){
          document.getElementById('logout-btn').style.display = 'none';
          document.getElementById('user-icon-btn').style.display = 'none';
          document.getElementById('login-btn').style.display = 'block';
          document.getElementById('signup-btn').style.display = 'block';

          google.colab.kernel.invokeFunction('notebook.logout', [], {});

        }

        const chatBox = document.getElementById('chat-box');
        const userInput = document.getElementById('user-input');

        function sendMessage() {
          console.log("HHIIIII from SENDMESSAGE");
          const userMsg = userInput.value.trim();
          if (userMsg === '') return;

          appendMessage(userMsg, 'user-msg');

          google.colab.kernel.invokeFunction('notebook.get_chatbot_response', [userMsg], {})
            .then((output) =>{
              const response = output.data['text/plain'];
              console.log("RESPONSE FROM CHATBOT: ", response);
              appendMessage(response, 'bot-msg');
            });
          userInput.value = '';
        }

        function appendMessage(message, className) {
          const messageDiv = document.createElement('div');
          messageDiv.className = className;
          messageDiv.textContent = message;
          chatBox.appendChild(messageDiv);
          chatBox.scrollTop = chatBox.scrollHeight;
        }



      </script>
    </head>
    <body>
      <div class="container-fluid p-0">
        <nav class="navbar navbar-expand-lg navbar-light bg-light">
          <span class="navbar-brand" href="#">
            <img src="#" alt="" />ClSearch
          </span>
          <button
            class="navbar-toggler"
            type="button"
            data-toggle="collapse"
            data-target="#navbarNavAltMarkup"
            aria-controls="navbarNavAltMarkup"
            aria-expanded="false"
            aria-label="Toggle navigation"
          >
            <span class="navbar-toggler-icon"></span>
          </button>
          <div class="collapse navbar-collapse" id="navbarNavAltMarkup">
            <div class="navbar-nav">
              <button
                class="nav-item nav-link active btn"
                onclick="showContent('home-content')"
              >
                Home <span class="sr-only">(current)</span>
              </button>
              <button
                class="nav-item nav-link btn"
                onclick="showContent('index-content')"
              >
                Index Management
              </button>
              <button
                class="nav-item nav-link btn"
                onclick="showContent('statistics-content'); showStatistics();"
              >
                Statistics
              </button>
            </div>
          </div>
          <button id="login-btn" class="nav-item nav-link btn" onclick="showContent('login-content')">Login</button>
          <button id="signup-btn" class="nav-item nav-link btn" onclick="showContent('signup-content')">SignUp</button>

          <button id="logout-btn" class="nav-item nav-link btn" style="display: none;" onclick="logout();">Logout</button>

          <div class="dropdown" style="position: relative;">
            <button id="user-icon-btn" class="nav-item nav-link btn" style="display: none;" onclick="toggleDropdown();">
              <svg xmlns="http://www.w3.org/2000/svg" width="16" height="16" fill="currentColor" class="bi bi-person" viewBox="0 0 16 16">
                <path d="M8 8a3 3 0 1 0 0-6 3 3 0 0 0 0 6m2-3a2 2 0 1 1-4 0 2 2 0 0 1 4 0m4 8c0 1-1 1-1 1H3s-1 0-1-1 1-4 6-4 6 3 6 4m-1-.004c-.001-.246-.154-.986-.832-1.664C11.516 10.68 10.289 10 8 10s-3.516.68-4.168 1.332c-.678.678-.83 1.418-.832 1.664z"/>
              </svg>
            </button>

            <!-- Dropdown Menu -->
            <div id="user-dropdown" class="dropdown-menu" style="display: none;">
              <button class="dropdown-item" onclick="showContent('user-statistics-content'); showUserStatistics();">User Statistics</button>
              <button class="dropdown-item" onclick="showContent('chatbot-content');">ChatBot</button>
              <button class="dropdown-item">Button3</button>
            </div>
          </div>

        </nav>

        <!-- Home Content Section -->
        <section class="content-section active-content" id="home-content">
          <div class="text-center display-4 my-5">
            <span style="color: #00695c; font-style: italic">ClSearch</span>
          </div>
          <form
            class="d-flex justify-content-center align-items-center vh-90 md-form form-sm active-cyan-2 mt-2"
            onsubmit="search(event)"
          >
            <div style="width: 100%">
              <input
                class="form-control form-control-sm w-75"
                style="margin: auto"
                id="search-input"
                type="text"
                placeholder="Search"
                aria-label="Search"
              />
              <div id="results-content">

                <!-- Add your content specific to Index Management here -->
              </div>
            </div>
          </form>
        </section>

        <!-- Index Management Content Section -->
        <section class="content-section" id="index-content">
          <div class="text-center display-4 my-5">
            <span style="color: #00695c; font-style: italic"
              >Index Management</span
            >

          </div>
          <div class="flex-container">
            <div class="table-container">
              <table class="table">
                <thead>
                  <tr>
                    <th scope="col">Words</th>
                    <th scope="col">Links</th>
                  </tr>
                </thead>
                <tbody id="index-table-body">
                  <!-- Rows will be dynamically added here -->
                </tbody>
              </table>
            </div>
          </div>
          <div class="button-container">
            <button class="btn btn-primary" id="addWordBtn">Add Word</button>
            <div id="addWordSection" style="display: none;">
              <div class="form-group">
                <label for="newWord">New Word:</label>
                <input type="text" class="form-control" id="newWord">
              </div>
              <div class="form-group">
                <label for="newLink">New Link:</label>
                <input type="text" class="form-control" id="newLink">
              </div>
              <button id="saveWordBtn" class="btn btn-success" style="margin-bottom: 10px;">Save</button>
              <button id="cancelBtn" class="btn btn-secondary" style="margin-bottom: 10px;">Cancel</button>
            </div>

            <button class="btn btn-primary remove-word-btn">Remove Word</button>
            <button class="btn btn-primary" id="addLinkBtn">Add Link</button>
            <div id="addLinkSection" style="display: none;">
              <div class="form-group">
                <label for="newLink">New Link:</label>
                <input type="text" class="form-control" id="newLink-addlink" placeholder="Enter new link">
              </div>
              <button type="button" class="btn btn-primary" id="saveLinkBtn" style="margin-bottom: 10px;">Save</button>
              <button type="button" class="btn btn-secondary" id="cancelLinkBtn" style="margin-bottom: 10px;">Cancel</button>
            </div>
          </div>
        </section>


        <section class="content-section" id="statistics-content">
          <div class="text-center display-4 my-5">
            <span style="color: #00695c; font-style: italic">Statistics</span>
          </div>
          <div id="statistics-chart"></div>

        </section>


        <!-- SignUp Content Section -->
        <section class="content-section" id="signup-content">
          <div class="text-center display-4 my-5">
            <span style="color: #00695c; font-style: italic">ClSearch</span>
          </div>

          <!-- SignUp Form -->
          <div class="container">
            <div class="row">
              <div class="col-md-6">
                <form>
                  <div class="form-group">
                    <label for="signup-username">Username</label>
                    <input type="text" class="form-control" id="signup-username" placeholder="Enter username">
                  </div>
                  <div class="form-group">
                    <label for="signup-password">Password</label>
                    <input type="password" class="form-control" id="signup-password" placeholder="Password">
                  </div>
                  <div class="form-group">
                    <label for="signup-security-question">Security Question</label>
                    <input type="text" class="form-control" id="signup-security-question" placeholder="Your favorite cloud company">
                  </div>
                  <button type="button" class="btn btn-primary" id="signup-btn" onclick="signup();">SignUp</button>
                  <div id="signup-message"></div>
                </form>
              </div>
            </div>
        </section>

        <!-- Login Content Section -->
        <section class="content-section" id="login-content">
          <div class="text-center display-4 my-5">
            <span style="color: #00695c; font-style: italic">ClSearch</span>
          </div>

          <!-- Login Form -->
          <div class="container">
            <div class="row">
              <div class="col-md-6">
                <form>
                  <div class="form-group">
                    <label for="login-username">Username</label>
                    <input type="text" class="form-control" id="login-username" placeholder="Enter username">
                  </div>
                  <div class="form-group">
                    <label for="login-password">Password</label>
                    <input type="password" class="form-control" id="login-password" placeholder="Password">
                  </div>
                  <button type="button" class="btn btn-primary" id="signup-btn" onclick="login();">Login</button>
                  <div id="login-message"></div>

                </form>
              </div>
            </div>
        </section>

        <!-- User Statistics Content Section -->
        <section class="content-section" id="user-statistics-content">
          <div class="text-center display-4 my-5">
            <span style="color: #00695c; font-style: italic">User Statistics</span>
          </div>

          <!-- Login Form -->
          <div id="user-statistics-chart" class="charts-container">
          </div>
        </section>

        <!-- Chatbot Content Section -->
        <section class="content-section" id="chatbot-content">
          <div class="text-center display-4 my-5">
            <span style="color: #00695c; font-style: italic">ChatBot</span>
          </div>

          <!-- Chat Form -->
          <div class="chat-container">
            <div class="chat-box" id="chat-box">
              <div class="bot-msg">Hello! I'm ChatBot. How can I assist you today?</div>
            </div>
            <input type="text" id="user-input" placeholder="Type your message here..." autocomplete="off" style="width: 100%; padding: 10px; box-sizing: border-box; border: 1px solid #ccc; border-top: none;">
            <button type="button" class="btn btn-primary" onclick="sendMessage();">Send</button>

          </div>
        </section>

      </div>

    </body>
  </html>

"""


index = {}
index = create_index(index)
upload_index_to_firestore(index)







index_data = extract_index_data_from_firestore()

index_table_rows = generate_index_table_rows(index_data)
html = html.replace("<!-- Rows will be dynamically added here -->", index_table_rows)

def handle_query(query):
    global user_query, index
    user_query = remove_stop_words(query)

    temp_index = []
    temp_index2 = []
    buffer1, buffer2 = [], []
    words = user_query.lower().split(" ")
    no_duplicates = list(set(words))
    index, temp_index, temp_index2 = update_index(no_duplicates, index)

    results = []


    javascript_resetConten = f"""
                <script>
                  resetResultContent();
                </script>
                """

    display(HTML(javascript_resetConten))






    for link in temp_index2:
      title = extract_link_title(link)
      count = count_query_occurrences(link, user_query)
      results.append({"title": title, "link": link, "occurrences": count})

    occurrences = [result["occurrences"] for result in results]

    rank = calculate_rank(occurrences)
    sorted_rank = sorted(rank, reverse=True)


    # Sort results based on the count of occurrences in descending order
    results.sort(key=lambda x: x["occurrences"], reverse=True)

    result_json = json.dumps(results)
    rank_json = json.dumps(sorted_rank)
    javascript = f"""
                <script>
                  updateResultsContent({result_json}, {rank_json});
                </script>
                """
    display(HTML(javascript))




    if len(temp_index2) == 0:
      display(HTML('<script>displayNoResults();</script>'))


    index, buffer1, buffer2 = update_index(no_duplicates, index)
    if len(temp_index2) != 0:
      upload_index_to_firestore(index)


    # Reference the collection in Firestore
    users_collection = db.collection('users')

    # Query users who are logged in
    query = users_collection.where('isLoggedIn', '==', 1).get()
    logged_in_users = [doc.id for doc in query]

    logged_user = logged_in_users[0]

    user_ref = db.collection('users').document(logged_user)
    user_doc = user_ref.get()
    if user_doc.exists:
      user_data = user_doc.to_dict()
      user_id = user_data.get('user_id')

      if user_id:
        user_sessions_collection = db.collection('user_sessions')
        user_session_ref = user_sessions_collection.document(user_id)
        user_session_doc = user_session_ref.get()
        if user_session_doc.exists:
          user_session_data = user_session_doc.to_dict()
          queries = user_session_data.get('queries', [])
          answers = user_session_data.get('answers', [])
          created_at = user_session_data.get('created_at', None)

          # Append the new query to the queries array
          queries.append(user_query)

          # Append the results to the results array
          answers.extend(results)

          created_at = datetime.datetime.now()

          # Update the user session document
          user_session_ref.update({
              'queries': queries,
              'answers': answers,
              'created_at': created_at
            })








# Register the Python function to be invoked from JavaScript
output.register_callback('notebook.query', handle_query)

# Display the HTML with JavaScript code
display(HTML(html))


Words,Links
accuracy,https://www.ibm.com/cloud/case-studies
add,https://www.ibm.com/products/satellite
ai,https://www.ibm.com/products/vmwarehttps://www.ibm.com/cloud/solutionshttps://www.ibm.com/blog/5-things-to-know-ibm-clouds-mission-to-accelerate-innovation-for-clients/?utm_medium=OSocial&utm_source=Linkedin&utm_content=FSCWW&utm_term=20CN4&utm_id=Cloud23-2023-03-08-Social-Post-IBM-Cloud-Mission&social_post=sf180688100&sf180688100=1https://www.ibm.com/cloud/financial-serviceshttps://www.ibm.com/blog/ubotica-partners-with-ibm-for-one-click-deployment-of-space-ai-applications/https://www.kuppingercole.com/research/lc80848/cloud-native-application-protection-platforms-cnapphttps://www.ibm.com/cloud/case-studieshttps://www.ibm.com/products/satellitehttps://www.ibm.com/cloud/saphttps://www.ibm.com/blog/ibm-cloud-delivers-enterprise-sovereign-cloud-capabilities/https://c212.net/c/link/?t=0&l=en&o=3120992-1&h=3670330057&u=https%3A%2F%2Fnewsroom.ibm.com%2F2020-07-22-IBM-and-Bank-of-America-Advance-IBM-Cloud-for-Financial-Services-BNP-Paribas-Joins-as-Anchor-Client-in-Europe&a=BNP+Paribashttps://www.ibm.com/products/cloud-object-storagehttps://www.ibm.com/downloads/cas/RGKYOOKBhttps://newsroom.ibm.com/How-IBM-is-Helping-Clients-Deploy-Foundation-Models-and-AI-Workloads-with-New-GPU-Offering-on-IBM-Cloud
application,https://www.ibm.com/cloud/solutionshttps://www.ibm.com/products/vmwarehttps://www.ibm.com/blog/ubotica-partners-with-ibm-for-one-click-deployment-of-space-ai-applications/https://www.kuppingercole.com/research/lc80848/cloud-native-application-protection-platforms-cnapphttps://www.ibm.com/downloads/cas/MZA4QK5Mhttps://www.ibm.com/cloud/case-studieshttps://www.ibm.com/products/satellitehttps://www.ibm.com/cloud/sap
business,https://www.ibm.com/products/vmwarehttps://www.ibm.com/cloud/solutionshttps://www.ibm.com/blog/5-things-to-know-ibm-clouds-mission-to-accelerate-innovation-for-clients/?utm_medium=OSocial&utm_source=Linkedin&utm_content=FSCWW&utm_term=20CN4&utm_id=Cloud23-2023-03-08-Social-Post-IBM-Cloud-Mission&social_post=sf180688100&sf180688100=1https://www.ibm.com/cloud/financial-serviceshttps://www.ibm.com/blog/ubotica-partners-with-ibm-for-one-click-deployment-of-space-ai-applications/https://www.kuppingercole.com/research/lc80848/cloud-native-application-protection-platforms-cnapphttps://www.businesswire.com/news/home/20221117005261/en/IDC-Introduces-the-Trust-Perception-Index-Measuring-Trust-Perceptions-for-the-Top-Six-Cloud-Providershttps://www.ibm.com/cloud/case-studieshttps://www.ibm.com/products/satellitehttps://www.ibm.com/cloud/saphttps://www.ibm.com/blog/ibm-cloud-delivers-enterprise-sovereign-cloud-capabilities/https://c212.net/c/link/?t=0&l=en&o=3120992-1&h=3670330057&u=https%3A%2F%2Fnewsroom.ibm.com%2F2020-07-22-IBM-and-Bank-of-America-Advance-IBM-Cloud-for-Financial-Services-BNP-Paribas-Joins-as-Anchor-Client-in-Europe&a=BNP+Paribashttps://www.ibm.com/products/cloud-object-storagehttps://newsroom.ibm.com/How-IBM-is-Helping-Clients-Deploy-Foundation-Models-and-AI-Workloads-with-New-GPU-Offering-on-IBM-Cloud
cloud,https://www.ibm.com/products/vmwarehttps://www.ibm.com/cloud/solutionshttps://www.ibm.com/blog/5-things-to-know-ibm-clouds-mission-to-accelerate-innovation-for-clients/?utm_medium=OSocial&utm_source=Linkedin&utm_content=FSCWW&utm_term=20CN4&utm_id=Cloud23-2023-03-08-Social-Post-IBM-Cloud-Mission&social_post=sf180688100&sf180688100=1https://www.ibm.com/cloud/financial-serviceshttps://www.ibm.com/blog/ubotica-partners-with-ibm-for-one-click-deployment-of-space-ai-applications/https://www.kuppingercole.com/research/lc80848/cloud-native-application-protection-platforms-cnapphttps://www.businesswire.com/news/home/20221117005261/en/IDC-Introduces-the-Trust-Perception-Index-Measuring-Trust-Perceptions-for-the-Top-Six-Cloud-Providershttps://www.ibm.com/cloud/case-studieshttps://www.ibm.com/products/satellitehttps://www.ibm.com/cloud/data-centershttps://www.ibm.com/cloud/saphttps://www.ibm.com/blog/ibm